In [6]:
# https://mne.tools/stable/auto_tutorials/forward/30_forward.html

import mne
from mne.datasets import sample
data_path = sample.data_path()

# the raw file containing the channel location + types
# sample_dir = data_path / 'MEG' / 'sample'
# raw_fname = sample_dir / 'sample_audvis_raw.fif'
# sample_dir = '/home/zhibinz2/Documents/GitHub/CAMCAN_MEG_100/CC120319/rest/'
# raw_fname = sample_dir / 'rest_raw.fif'
# raw_fname = '../CAMCAN_MEG_100/CC120319/rest/rest_raw.fif'
raw_fname = '../CAMCAN_MEG_100/CC110033/task/task_raw.fif'
# The paths to Freesurfer reconstructions
subjects_dir = data_path / 'subjects'
subject = 'fsaverage' # change it to use freesurfer's bem

In [7]:
# trans = 'sample_fsaverage_manual_trans.fif'
trans = './test_scripts/CAMCAN_fsaverage_trans.fif'
info = mne.io.read_info(raw_fname)

    Read a total of 8 projection items:
        mag_ssp_upright.fif : PCA-mags-v1 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v2 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v3 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v4 (1 x 306)  idle
        mag_ssp_upright.fif : PCA-mags-v5 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v1 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v2 (1 x 306)  idle
        grad_ssp_upright.fif : PCA-grad-v3 (1 x 306)  idle


In [8]:
# Compute Source Space
# https://mne.tools/0.19/generated/mne.setup_source_space.html
# The spacing to use. Can be 'ico#' for a recursively subdivided icosahedron, 
# 'oct#' for a recursively subdivided octahedron, 
# 'all' for all points, or an integer to use appoximate distance-based spacing (in mm).
src = mne.setup_source_space(subject, spacing='ico4', add_dist='patch',
                             subjects_dir=subjects_dir)
# oct8 takes 2.5 min produces 65538 source
# ico7 = 'all'

Setting up the source space with the following parameters:

SUBJECTS_DIR = C:\Users\zhouz\mne_data\MNE-sample-data\subjects
Subject      = fsaverage
Surface      = white
Icosahedron subdivision grade 4

>>> 1. Creating the source space...

Doing the icosahedral vertex picking...
Loading C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.white...
Mapping lh fsaverage -> ico (4) ...
    Triangle neighbors and vertex normals...
Loading geometry from C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\surf\lh.sphere...
Setting up the triangulation for the decimated surface...
loaded lh.white 2562/163842 selected to source space (ico = 4)

Loading C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\surf\rh.white...
Mapping rh fsaverage -> ico (4) ...
    Triangle neighbors and vertex normals...
Loading geometry from C:\Users\zhouz\mne_data\MNE-sample-data\subjects\fsaverage\surf\rh.sphere...
Setting up the triangulation for the decimated surface...
loaded rh.wh

In [9]:
# compute forward solution
conductivity = (0.3,)  # for single layer
# conductivity = (0.3, 0.006, 0.3)  # for three layers
model = mne.make_bem_model(subject='fsaverage', ico=4,
                           conductivity=conductivity,
                           subjects_dir=subjects_dir)
bem = mne.make_bem_solution(model)

Creating the BEM geometry...
Going from 5th to 4th subdivision of an icosahedron (n_tri: 20480 -> 5120)
inner skull CM is  -0.53 -21.10   6.21 mm
Surfaces passed the basic topology checks.
Complete.

Homogeneous model surface loaded.
Computing the linear collocation solution...
    Matrix coefficients...
        inner skull (2562) -> inner skull (2562) ...
    Inverting the coefficient matrix...
Solution ready.
BEM geometry computations complete.


In [10]:
fwd = mne.make_forward_solution(raw_fname, trans=trans, src=src, bem=bem,
                                meg=True, eeg=False, mindist=5.0, n_jobs=None,
                                verbose=True)
print(fwd)
#  1m17s

Source space          : <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=2562>, <surface (rh), n_vertices=163842, n_used=2562>] MRI (surface RAS) coords, subject 'fsaverage', ~29.5 MB>
MRI -> head transform : test_scripts\CAMCAN_fsaverage_trans.fif
Measurement data      : task_raw.fif
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 2 source spaces a total of 5124 active source locations

Coordinate transformation: MRI (surface RAS) -> head
     0.999927 -0.009207 -0.007775      -4.28 mm
     0.009519  0.999110  0.041099      24.90 mm
     0.007390 -0.041170  0.999125      31.33 mm
     0.000000  0.000000  0.000000       1.00

Read 306 MEG channels from info
105 coil definitions read
Coordinate transformation: MEG device -> head
     0.998429 -0.050574  0.024122       2.53 mm
     0.052468  0.994938 -0.085709     -15.30 mm
    -0.019665  0.086840  0.996028      39.95 mm
     0.000000  0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found    0/1953 points outside using solid angles
    Total 2562/2562 points inside the surface
Interior check completed in 859.2 ms
Checking surface interior status for 2562 points...
    Found  561/2562 points inside  an interior sphere of radius   47.7 mm
    Found    0/2562 points outside an exterior sphere of radius   98.3 mm
    Found    0/2001 points outside using surface Qhull


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


    Found    0/2001 points outside using solid angles
    Total 2562/2562 points inside the surface
Interior check completed in 849.7 ms

Checking surface interior status for 306 points...
    Found   0/306 points inside  an interior sphere of radius   47.7 mm
    Found 306/306 points outside an exterior sphere of radius   98.3 mm
    Found   0/  0 points outside using surface Qhull
    Found   0/  0 points outside using solid angles
    Total 0/306 points inside the surface
Interior check completed in 48.6 ms

Composing the field computation matrix...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing MEG at 5124 source locations (free orientations)...


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s finished



Finished.
<Forward | MEG channels: 306 | EEG channels: 0 | Source space: Surface with 5124 vertices | Source orientation: Free>


In [20]:
print(f'Before: {src}')
print(f'After:  {fwd["src"]}')

Before: <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=40962>, <surface (rh), n_vertices=163842, n_used=40962>] MRI (surface RAS) coords, subject 'fsaverage', ~42.6 MB>
After:  <SourceSpaces: [<surface (lh), n_vertices=163842, n_used=40962>, <surface (rh), n_vertices=163842, n_used=40962>] head coords, subject 'fsaverage', ~42.6 MB>


In [11]:
fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True)
leadfield = fwd_fixed['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

    Average patch normals will be employed in the rotation to the local surface coordinates....
    Converting to surface-based source orientations...
    [done]
Leadfield size : 306 sensors x 5124 dipoles


In [12]:
from hdf5storage import loadmat, savemat 
outdict=dict()
outdict['leadfield']=leadfield
outdict['source_nn']=fwd_fixed['source_nn']
outdict['source_rr']=fwd_fixed['source_rr']
savemat('leadfield_nn_rr',outdict)

In [17]:
mne.write_forward_solution('simple_fsaverage_fixed_fwd.fif', fwd_fixed, overwrite=False, verbose=None)

    Write a source space...
    [done]
    Write a source space...
    [done]
    2 source spaces written


C:\Users\zhouz\AppData\Local\Temp\ipykernel_24484\3691629218.py:1: RuntimeWarning: This forward solution is based on a forward solution with free orientation. The original forward solution is stored on disk in X/Y/Z RAS coordinates. Any transformation (surface orientation or fixed orientation) will be reverted. To reapply any transformation to the forward operator please apply convert_forward_solution after reading the forward solution with read_forward_solution.
  mne.write_forward_solution('simple_fsaverage_fixed_fwd.fif', fwd_fixed, overwrite=False, verbose=None)


In [19]:
# forward = mne.read_forward_solution('simple_fsaverage_fixed_fwd.fif',verbose=None)

Reading forward solution from /home/zhibinz2/Documents/GitHub/MEG_EEG_Source_Localization/simple_fsaverage_fwd.fif...
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    Reading a source space...
    Computing patch statistics...
    Patch information added...
    [done]
    2 source spaces read
    Desired named matrix (kind = 3523) not available
    Read MEG forward solution (516 sources, 306 channels, free orientations)
    Source spaces transformed to the forward solution coordinate frame
